In [1]:
from collections import defaultdict
import importlib
import itertools
import joblib
import lightgbm as lgb
import os
import numpy as np

import sys
sys.path.append("../src/utils")
from annovar_utils import *
from variant_utils import Variant
from comparison_utils import *

import evaluation_utils
importlib.reload(evaluation_utils)
from evaluation_utils import *

In [2]:
DDD_annovar_output_path = "/cluster/u/rrastogi/ECNN/dataset/DDD/annovar_output"
DDD_results_path = "/cluster/u/rrastogi/ECNN/exomes/disease"

In [3]:
xcap_positive_variants, xcap_negative_variants = get_xcap_train_variants()
comp_positive_variants = get_competitors_positive_variants()

In [4]:
def get_causal_variants(filepath):
    variants = []
    for i, line in enumerate(open(filepath)):
        causal = get_value_from_line(line, "LABEL", lambda x: int(x))
        if causal:
            variants.append((get_variant_from_line(line), i))
    return variants

def get_benign_variants(filepath):
    variants = []
    for i, line in enumerate(open(filepath)):
        causal = get_value_from_line(line, "LABEL", lambda x: int(x))
        if not causal:
            variants.append((get_variant_from_line(line), i))
    return variants

def get_num_variants(filepath):
    return sum([1 for line in open(filepath)])

def get_filtered_patients():
    patients, directories = [], []
    for pt in os.listdir(DDD_annovar_output_path):
        pt_dir = os.path.join(DDD_annovar_output_path, pt)
        if not os.path.isdir(pt_dir) or not pt.startswith("DDD"):
            continue
    
        coding_filtered_path = os.path.join(pt_dir, "{}_filtered.exonic_variant_function".format(pt))
        stopgain_filtered_path = os.path.join(pt_dir, "{}_stopgain_filtered.tsv".format(pt))
    
        causal_variants = get_causal_variants(coding_filtered_path)
        causal_stopgains = get_causal_variants(stopgain_filtered_path)
        
        if len(causal_variants) != 1 or len(causal_stopgains) != 1:
            continue
    
        causal_stopgain, _ = causal_stopgains[0]
        if causal_stopgain in comp_positive_variants:
            continue
        
        patients.append(pt)
        directories.append(pt_dir)
    return patients, directories

## Get classifier predictions on filtered patients

In [5]:
aloft_results_dir = "/cluster/u/rrastogi/ECNN/exomes/disease/aloft"

def get_aloft_results(pt, labels):
    scores = defaultdict(list)
    zygosities = [False for _ in range(len(labels))]
    
    results_path = os.path.join(aloft_results_dir, "{}_2".format(pt), "aloft_output", "hg19.aloft.lof.results")
    for line in open(results_path):
        if line.startswith("#"):
            continue
        row = line.strip().split('\t')
        index = get_value_from_info(row[7], "PRE_LIFTOVER_INDEX", int)
        score, zygosity = get_prediction_from_aloft_row(row)
        scores[index].append(score)
        zygosities[index] = zygosity
        
    averaged_scores = [0.0 for _ in range(len(labels))]
    for index, score_l in scores.items():
        averaged_scores[index] = np.mean(score_l)
    
    return averaged_scores, zygosities

In [6]:
mutpred_results_dir = "/cluster/u/rrastogi/ECNN/exomes/disease/mutpred"

def get_mutpred_results(pt, original_tsv):
    old_ordering = {}
    for i, line in enumerate(open(original_tsv)):
        v = get_variant_from_line(line)
        old_ordering[v] = i
    
    new_ordering = {}
    new_tsv = os.path.join(mutpred_results_dir, pt, "subset.tsv")
    for i, line in enumerate(open(new_tsv)):
        v = get_variant_from_line(line)
        new_ordering[i] = v
    
    ordered_scores = [0 for _ in range(len(old_ordering))]
    results_path = os.path.join(mutpred_results_dir, pt, "results_output.txt")
    for i, line in enumerate(open(results_path)):
        row = line.strip().split('|')
        score = float(row[1])
        associated_v = new_ordering[i]
        ordered_index = old_ordering[associated_v]
        ordered_scores[ordered_index] = score
    return ordered_scores

In [7]:
xcap_results_dir = "/cluster/u/rrastogi/ECNN/exomes/disease/xcap"
xcap_clf = lgb.Booster(model_file="/cluster/u/rrastogi/ECNN/results/d_original/xcap/featurize_0520/clf_augment.mdl")

def get_xcap_results(pt, original_tsv):
    features_path = os.path.join(xcap_results_dir, pt, "{}.features".format(pt))
    labels, predictions = get_xcap_predictions(xcap_clf, features_path)
    return predictions

In [8]:
WGP_prediction_fxns = {
    "CADD": get_cadd_predictions,
    "DANN": get_dann_predictions,
    "Eigen": get_eigen_predictions,
}

def get_wgp_results(pt, wgp_name):
    wgp_input_fname = os.path.join("/cluster/u/rrastogi/ECNN/dataset/DDD/annovar_output", pt, "{}_stopgain_filtered.tsv".format(pt))
    wgp_results_fname = os.path.join("/cluster/u/rrastogi/ECNN/exomes/disease/wgp", pt, "avinput.hg38_multianno.txt")
    
    fxn = WGP_prediction_fxns[wgp_name]
    _, predictions = fxn(wgp_input_fname, wgp_results_fname)
    return predictions

In [9]:
load_aloft_percentiles()
load_mutpred_percentiles()
load_xcap_percentiles(xcap_clf)
load_cadd_percentiles()
load_dann_percentiles()
load_eigen_percentiles()

In [14]:
filtered_patients, filtered_directories = get_filtered_patients()
for pt, pt_dir in zip(filtered_patients, filtered_directories):
    stopgain_path = os.path.join(pt_dir, "{}_stopgain_filtered.tsv".format(pt))
    num_variants = get_num_variants(stopgain_path)
    causal_variant, causal_index = get_causal_variants(stopgain_path)[0]
    labels = [0 if i != causal_index else 1 for i in range(num_variants)]
    
    aloft_scores, zygosities = get_aloft_results(pt, labels)
    causal_a_score = aloft_scores[labels.index(1)]
    causal_a_percentile = get_aloft_percentile(causal_a_score)
    
    mutpred_scores = get_mutpred_results(pt, stopgain_path)
    causal_m_score = mutpred_scores[labels.index(1)]
    causal_m_percentile = get_mutpred_percentile(causal_m_score)
    
    xcap_scores = get_xcap_results(pt, stopgain_path)
    causal_x_score = xcap_scores[labels.index(1)]
    causal_x_percentile = get_xcap_percentile(causal_x_score)
    
    cadd_scores = get_wgp_results(pt, "CADD")
    causal_c_score = cadd_scores[labels.index(1)]
    causal_c_percentile = get_cadd_percentile(causal_c_score)
    
    dann_scores = get_wgp_results(pt, "DANN")
    causal_d_score = dann_scores[labels.index(1)]
    causal_d_percentile = get_dann_percentile(causal_d_score)
    
    eigen_scores = get_wgp_results(pt, "Eigen")
    causal_e_score = eigen_scores[labels.index(1)]
    causal_e_percentile = get_eigen_percentile(causal_e_score)
    
    
    print("Patient: {}".format(pt))
    print("Causal variant: {}".format(causal_variant))
    print("X-CAP: causal={}, %={}".format(round(causal_x_score, 3), round(causal_x_percentile, 2)))
    print("MutPred-LoF: causal={}, %={}".format(round(causal_m_score, 3), round(causal_m_percentile, 2)))
    print("ALoFT: causal={}, %={}".format(round(causal_a_score, 3), round(causal_a_percentile, 2)))
    print("CADD: causal={}, %={}".format(round(causal_c_score, 3), round(causal_c_percentile, 2)))
    print("DANN: causal={}, %={}".format(round(causal_d_score, 3), round(causal_d_percentile, 2)))
    print("Eigen: causal={}, %={}".format(round(causal_e_score, 3), round(causal_e_percentile, 2)))
    print("")

Patient: DDDP108441
Causal variant: Chr: 3, pos: 70977705, ref: G, alt: A
X-CAP: causal=0.687, %=89.44
MutPred-LoF: causal=0.586, %=87.7
ALoFT: causal=0.584, %=89.38
CADD: causal=44.0, %=89.21
DANN: causal=0.998, %=81.3
Eigen: causal=1.149, %=95.14

Patient: DDDP108556
Causal variant: Chr: X, pos: 71137818, ref: C, alt: A
X-CAP: causal=0.834, %=90.82
MutPred-LoF: causal=0.348, %=26.6
ALoFT: causal=0.792, %=93.34
CADD: causal=35.0, %=23.73
DANN: causal=0.996, %=52.02
Eigen: causal=-0.589, %=8.93

Patient: DDDP108105
Causal variant: Chr: 2, pos: 199328709, ref: G, alt: A
X-CAP: causal=1.0, %=98.81
MutPred-LoF: causal=0.533, %=77.3
ALoFT: causal=0.926, %=96.37
CADD: causal=38.0, %=57.24
DANN: causal=0.997, %=64.07
Eigen: causal=0.85, %=69.97

Patient: DDDP109873
Causal variant: Chr: 22, pos: 41177292, ref: C, alt: T
X-CAP: causal=0.827, %=90.78
MutPred-LoF: causal=0.713, %=98.9
ALoFT: causal=0.973, %=98.82
CADD: causal=38.0, %=57.24
DANN: causal=0.994, %=44.11
Eigen: causal=0.548, %=46.49